In [29]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.callbacks import ModelCheckpoint

In [40]:
path = 'weights/rock_paper_scissor/checkpoint'
callbacks = ModelCheckpoint(
    filepath=path,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

In [31]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32,(3,3),activation="relu",input_shape=(128,128,3)))
model.add(keras.layers.MaxPool2D((2,2)))
model.add(keras.layers.Conv2D(64,(3,3),activation="relu"))
model.add(keras.layers.MaxPool2D((2,2)))
model.add(keras.layers.Conv2D(128,(3,3),activation="relu"))
model.add(keras.layers.MaxPool2D((2,2)))
model.add(keras.layers.Conv2D(128,(3,3),activation="relu"))
model.add(keras.layers.MaxPool2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dense(3, activation="softmax"))
model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['acc'])

In [70]:
TRAINING_DIR = "data/rock_paper_scissor/training"
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,batch_size=32,class_mode="categorical",target_size = (128,128))

VALIDATION_DIR = "data/rock_paper_scissor/testing"
validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,batch_size=32,class_mode="categorical",target_size = (128,128))

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


In [59]:
history = model.fit(train_generator,
                              epochs=15,
                              verbose=1,
                              validation_data=validation_generator,
                              validation_steps=3,
                              steps_per_epoch=20,
                              callbacks=[callbacks])

Epoch 1/15
20/20 [==============================] - 13s 662ms/step - loss: 0.3078 - acc: 0.8875 - val_loss: 0.0141 - val_acc: 1.0000
Epoch 2/15
20/20 [==============================] - 13s 666ms/step - loss: 0.2786 - acc: 0.8938 - val_loss: 0.6149 - val_acc: 0.8333
Epoch 3/15
20/20 [==============================] - 13s 665ms/step - loss: 0.2273 - acc: 0.9172 - val_loss: 0.0266 - val_acc: 1.0000
Epoch 4/15
20/20 [==============================] - 14s 688ms/step - loss: 0.2622 - acc: 0.9000 - val_loss: 0.0114 - val_acc: 1.0000
Epoch 5/15
20/20 [==============================] - 15s 728ms/step - loss: 0.2199 - acc: 0.9156 - val_loss: 0.1272 - val_acc: 0.8333
Epoch 6/15
20/20 [==============================] - 14s 704ms/step - loss: 0.1848 - acc: 0.9406 - val_loss: 1.0510 - val_acc: 0.6667
Epoch 7/15
20/20 [==============================] - 15s 732ms/step - loss: 0.2102 - acc: 0.9312 - val_loss: 0.2769 - val_acc: 0.8333
Epoch 8/15
20/20 [==============================] - 13s 667ms/step - 

In [54]:
model.load_weights(path)

In [80]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
unshuffle_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,batch_size=32,class_mode="categorical",target_size = (128,128), shuffle=False)
Y_pred = model.predict(unshuffle_generator)
y_pred = np.argmax(Y_pred, axis=1)

In [81]:
print('Confusion Matrix')
print(confusion_matrix(unshuffle_generator.classes, y_pred))

Confusion Matrix
[[108   7   9]
 [  0 124   0]
 [  0   0 124]]


In [82]:
print('Classification Report')
target_names = list(unshuffle_generator.class_indices.keys())
print(classification_report(unshuffle_generator.classes, y_pred, target_names=target_names))

Classification Report
              precision    recall  f1-score   support

       paper       1.00      0.87      0.93       124
        rock       0.95      1.00      0.97       124
    scissors       0.93      1.00      0.96       124

    accuracy                           0.96       372
   macro avg       0.96      0.96      0.96       372
weighted avg       0.96      0.96      0.96       372

